# Wikipedia Notable Deaths

## Context

The...

## Objective

The...

## Data Dictionary
Variable: Description

### Data Collection
Scraped on 6/9/22 with scrapy.

Main site: https://en.wikipedia.org/wiki/Lists_of_deaths_by_year
Start_url: https://en.wikipedia.org/wiki/Deaths_in_January_1994
paginating through May, 2022 successfully.
Reference count scraped separately,due to pagination, to maintain original order in first dataset.
June, 20222 as the current month had inconsistent formatting and was scraped separately. 
~13,000 missing entries for reference count.

6/10/22 set out to id links with missing reference count to rescrape.

## Importing necessary libraries

In [1]:
# To limit number of threads in numpy and thereby prevent known dataleak associated with KMeans
# Note:  this cell must be run BEFORE installing numpy to have desired effect
import os

os.environ["OMP_NUM_THREADS"] = "1"

In [2]:
# To structure code automatically
%load_ext nb_black

# To import sqlite databases
import sqlite3 as sql

# To help with reading and manipulating data
import pandas as pd
import numpy as np

# To help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# To be used for data scaling
from sklearn.preprocessing import StandardScaler

# To compute distances
from scipy.spatial.distance import cdist, pdist

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To perform k-means clustering and compute silhouette scores
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# To visualize the elbow curve and silhouette scores
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from yellowbrick.style.palettes import PALETTES

# To perform hierarchical clustering, compute cophenetic correlation, and create dendrograms
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet

# To supress scientific notations for a dataframe
pd.set_option("display.float_format", lambda x: "%.3f" % x)

# To supress warnings
import warnings

warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)
sns.set_theme()
sns.set_palette(
    (
        "midnightblue",
        "goldenrod",
        "maroon",
        "darkolivegreen",
        "cadetblue",
        "tab:purple",
        "yellowgreen",
    )
)
plt.rc("font", size=12)
plt.rc("axes", titlesize=15)
plt.rc("axes", labelsize=14)
plt.rc("xtick", labelsize=13)
plt.rc("ytick", labelsize=13)
plt.rc("legend", fontsize=13)
plt.rc("legend", fontsize=14)
plt.rc("figure", titlesize=16)

<IPython.core.display.Javascript object>

## Data Overview

### Reading, sampling, and checking data shape

### January 1994 through May 2022 Data (without reference counts)

In [3]:
# Reading the wp_deaths_94_to_22 dataset from sql db and table
conn = sql.connect("wp_deaths_94_to_22.db")
raw_94_to_22 = pd.read_sql("SELECT * FROM wp_deaths_94_to_22", conn)

# Making a working copy
df_94_to_22 = raw_94_to_22.copy()

# Checking the shape
print(f"There are {df_94_to_22.shape[0]} rows and {df_94_to_22.shape[1]} columns.")

# Checking first 2 rows of the data
df_94_to_22.head(2)

There are 133769 rows and 5 columns.


,month_year,day,name,info,link
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer)
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty


<IPython.core.display.Javascript object>

In [4]:
# Checking last 2 rows of the data
df_94_to_22.tail(2)

,month_year,day,name,info,link
133767,May 2022,31,Dave Smith,", 72, American sound engineer, founder of Sequential.",https://en.wikipedia.org/wiki/Dave_Smith_(engineer)
133768,May 2022,31,Wang Zherong,", 86, Chinese tank designer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Wang_Zherong


<IPython.core.display.Javascript object>

In [5]:
# Checking a sample of the data
df_94_to_22.sample(5)

,month_year,day,name,info,link
32216,November 2006,24,Walter Booker,", 72, American jazz bassist (Cannonball Adderley Quintet), cardiac arrest.",https://en.wikipedia.org/wiki/Walter_Booker
30928,June 2006,18,Chris and Cru Kahui,", 3-months, New Zealand child homicide victims.",https://en.wikipedia.org/wiki/Chris_and_Cru_Kahui
13851,June 1999,7,Joseph Vandernoot,", 84, British conductor.",https://en.wikipedia.org/wiki/Joseph_Vandernoot
84110,July 2016,18,Mubarak Begum,", 80, Indian playback singer.",https://en.wikipedia.org/wiki/Mubarak_Begum
72205,October 2014,19,John Holt,", 67, Jamaican singer (The Paragons) and songwriter (""The Tide Is High""), cancer.",https://en.wikipedia.org/wiki/John_Holt_(singer)


<IPython.core.display.Javascript object>

#### Observations:
- There are 133,769 rows and 5 columns in the data from January, 1994 through May, 2022.
- The number of references was scraped separately.

### January 1994 through May 2022 Reference Count Data

In [6]:
# Reading the wp_reference_counts_2 dataset from sql db and table
conn = sql.connect("wp_reference_counts_2.db")
raw_reference_counts = pd.read_sql("SELECT * FROM wp_reference_counts_2", conn)

# Making a working copy
df_reference_counts = raw_reference_counts.copy()

# Checking the shape
print(
    f"There are {df_reference_counts.shape[0]} rows and {df_reference_counts.shape[1]} columns."
)

# Checking first 2 rows of the data
df_reference_counts.head(2)

There are 120368 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/Lys_Gauty,5
1,https://en.wikipedia.org/wiki/William_Chappell_(dancer),21


<IPython.core.display.Javascript object>

In [7]:
# Checking last 2 rows of the data
df_reference_counts.tail(2)

,link,num_references
120366,https://en.wikipedia.org/wiki/Shirley_Thomas_(USC_professor),6
120367,https://en.wikipedia.org/wiki/James_Doohan,52


<IPython.core.display.Javascript object>

In [8]:
# Checking a sample of the data
df_reference_counts.sample(5)

,link,num_references
28815,https://en.wikipedia.org/wiki/Bruno_Galliker,5
90694,https://en.wikipedia.org/wiki/Ivor_G._Balding,5
65959,https://en.wikipedia.org/wiki/Roy_Bonisteel,9
33216,https://en.wikipedia.org/wiki/George_Clements,20
104370,https://en.wikipedia.org/wiki/Yank_Rachell,7


<IPython.core.display.Javascript object>

#### Observations:
- Here we see that there are ~13,000 fewer rows for the reference data, indicating some pages were not successfully scraped to obtain the number of references for the individual.
- After combining the three dataframes, we can reattempt a more brute force scraping of those pages, in order to obtain the missing information.

### June 2022 Data

In [9]:
# Reading the wp_deaths_June_2022 dataset from sql db and table
conn = sql.connect("wp_deaths_June_2022.db")
raw_June_2022 = pd.read_sql("SELECT * FROM wp_deaths_June_2022", conn)

# Making a working copy
df_June_2022 = raw_June_2022.copy()

# Checking the shape
print(f"There are {df_June_2022.shape[0]} rows and {df_June_2022.shape[1]} columns.")

# Checking first 2 rows of the data
df_June_2022.head(2)

There are 145 rows and 6 columns.


,month_year,day,name,info,link,num_references
0,June 2022,8,Mladen Frančić,", 67, Croatian football player and manager (Vrbovec, Podravina, Al-Watani Club).",https://en.wikipedia.org/wiki/Mladen_Fran%C4%8Di%C4%87,1
1,June 2022,6,Valery Ryumin,", 82, Russian cosmonaut (Soyuz 25, Soyuz 32, Soyuz 35).",https://en.wikipedia.org/wiki/Valery_Ryumin,2


<IPython.core.display.Javascript object>

#### Observations:
- The June, 2022 data does not follow the same row order as the previous dataframe, which was in order of day of the month. 
- For continuity, before concatinating the two dataframes, we can sort June, 2022 by day.

In [10]:
# Sorting by day
df_June_2022.sort_values(by="day", inplace=True)

# Re-checking first 2 rows of the data
df_June_2022.head(2)

,month_year,day,name,info,link,num_references
26,June 2022,1,Richard Oldcorn,", 84, English Olympic fencer (1964, 1968, 1972).",https://en.wikipedia.org/wiki/Richard_Oldcorn,5
20,June 2022,1,István Szőke,", 75, Hungarian footballer (Ferencváros, national team), stroke.",https://en.wikipedia.org/wiki/Istv%C3%A1n_Sz%C5%91ke,2


<IPython.core.display.Javascript object>

In [11]:
# Checking last 2 rows of the data
df_June_2022.tail(2)

,month_year,day,name,info,link,num_references
8,June 2022,9,Oleg Moliboga,", 69, Russian volleyball player, Olympic champion (1980) and coach.",https://en.wikipedia.org/wiki/Oleg_Moliboga,2
5,June 2022,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3


<IPython.core.display.Javascript object>

In [12]:
# Checking a sample of the data
df_June_2022.sample(5)

,month_year,day,name,info,link,num_references
77,June 2022,4,Goran Sankovič,", 42, Slovenian footballer (SK Slavia Prague, Panionios, national team).",https://en.wikipedia.org/wiki/Goran_Sankovi%C4%8D,4
30,June 2022,1,Aleksandr Berketov,", 46, Russian footballer (Rotor Volgograd, CSKA Moscow).",https://en.wikipedia.org/wiki/Aleksandr_Berketov,2
71,June 2022,3,Saafi Boulbaba,", 36, Tunisian footballer (Espérance Sportive de Tunis, A S Kasserine, MC El Eulma), traffic collision.",https://en.wikipedia.org/wiki/Saafi_Boulbaba,4
144,June 2022,8,Birkha Bahadur Muringla,", 79, Indian writer.",https://en.wikipedia.org/wiki/Birkha_Bahadur_Muringla,3
4,June 2022,8,Aslam Azad,", 73, Indian politician, Bihar MLC (2006–2012).",https://en.wikipedia.org/wiki/Aslam_Azad,5


<IPython.core.display.Javascript object>

#### Observations:
- Now, we are ready to combine the three dataframes.

## Combining Dataframes

### Adding Number of References to 1994 through May 2022 Data

In [13]:
# Adding num_references column to 1994 through May 2022 data
df_combined = pd.merge(df_94_to_22, df_reference_counts, how="left", on="link")

# Checking first 2 rows of the data
df_combined.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

### Adding June 2022 Data

In [14]:
# Adding Juned 2022 data
df_combined = pd.concat([df_combined, df_June_2022], ignore_index=True)

# Making a working copy
df = df_combined.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 133914 rows and 6 columns.


,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

In [15]:
# Checking last 2 rows of the data
df.tail(2)

,month_year,day,name,info,link,num_references
133912,June 2022,9,Oleg Moliboga,", 69, Russian volleyball player, Olympic champion (1980) and coach.",https://en.wikipedia.org/wiki/Oleg_Moliboga,2
133913,June 2022,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3


<IPython.core.display.Javascript object>

In [16]:
# Checking a sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references
12576,November 1998,16,Tyrone Delano Gilliam Jr.,", 32, American convicted murderer, execution by lethal injection.",https://en.wikipedia.org/wiki/Tyrone_Delano_Gilliam_Jr.,10
102984,March 2019,16,Gilbert Hottois,", 72, Belgian philosopher.",https://en.wikipedia.org/wiki/Gilbert_Hottois,2
5791,March 1996,23,"J. D. ""Jay"" Miller",", 73, American musician.",https://en.wikipedia.org/wiki/J._D._%22Jay%22_Miller,13
84606,August 2016,14,Neil Black,", 84, English oboist.",https://en.wikipedia.org/wiki/Neil_Black,8
44694,March 2010,11,Merlin Olsen,", 69, American football player (Los Angeles Rams), member of Pro Football Hall of Fame, and actor (, ), mesothelioma.",https://en.wikipedia.org/wiki/Merlin_Olsen,31


<IPython.core.display.Javascript object>

#### Confirming Correct Number of Resultant Entries

In [17]:
# Confirming correct number of total rows
df_94_to_22.shape[0] + df_June_2022.shape[0]

133914

<IPython.core.display.Javascript object>

#### Observations:
- We have successfully combined the three dataframes.
- Now, we can check for data types, duplicates, and missing values.

## Checking data types, duplicates, and null values

### Data types

In [18]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133914 entries, 0 to 133913
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   month_year      133914 non-null  object
 1   day             133914 non-null  object
 2   name            133903 non-null  object
 3   info            133914 non-null  object
 4   link            133914 non-null  object
 5   num_references  120637 non-null  object
dtypes: object(6)
memory usage: 6.1+ MB


<IPython.core.display.Javascript object>

#### Observations:
- There are 6 columns, all of object type.
- `name` and `num_references` both have missing values.
- The data is in a very raw format and there are other columns that have combined information that will need to be extracted.
- For now, we will leave all as object type.

### Duplicate Rows

In [19]:
# Checking duplicate rows
df.duplicated().sum()

9

<IPython.core.display.Javascript object>

#### Observations:
- There are 9 duplicate rows that we will drop now.

In [20]:
# Drop duplicate rows
df.drop_duplicates(inplace=True, ignore_index=True)

# Re-check shape
df.shape

(133905, 6)

<IPython.core.display.Javascript object>

### Missing Values

In [21]:
# Check percentage of null values by column
df.isnull().sum() / df.count() * 100

month_year        0.000
day               0.000
name              0.008
info              0.000
link              0.000
num_references   11.007
dtype: float64

<IPython.core.display.Javascript object>

In [22]:
# Checking number of missing values per row
df.isnull().sum(axis=1).value_counts()

0    120628
1     13266
2        11
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- The number of rows missing only 1 value appears consistent with our anticipated missing `num_references`.
- There are 11 rows that are each missing 2 values.  Let us take a closer look at these rows.

In [23]:
# Checking the rows that are missing values for 2 columns
missing_2 = df[df.isnull().sum(axis=1) == 2]
missing_2

,month_year,day,name,info,link,num_references
18937,August 2001,11,None,"Kevin Kowalcyk, 2, known for eating a hamburger contaminated with E. coli O157:H7.",https://en.wikipedia.orgNone,NaN
24985,January 2004,22,None,"Vincent Palmer, 37, British criminal.",https://en.wikipedia.orgNone,NaN
27458,March 2005,1,None,"Barry Stigler, 57, American voice actor.",https://en.wikipedia.orgNone,NaN
34077,July 2007,11,None,"Nana Gualdi, 75, German singer and actress.",https://en.wikipedia.orgNone,NaN
35097,November 2007,11,None,,https://en.wikipedia.orgNone,NaN
41075,May 2009,18,None,Either killed in a missile attack or shot:\n,https://en.wikipedia.orgNone,NaN
64771,September 2013,29,None,"Scott Workman, 47, American stuntman (, , ), cancer.",https://en.wikipedia.orgNone,NaN
76024,April 2015,29,None,Notable convicted drug traffickers executed by Indonesian firing squad:\n,https://en.wikipedia.orgNone,NaN
105871,August 2019,2,None,"Japanese convicted murderers, executed by hanging.\n",https://en.wikipedia.orgNone,NaN
106617,September 2019,12,None,"Thami Shobede, 31, Singer Songwriter",https://en.wikipedia.orgNone,NaN


<IPython.core.display.Javascript object>

#### Observations:
- We can see that multiple rows are missing `name`, however the name is contained in `info`, which we can extract later.  
- The missing link itself is not of concern as it serves only as a means by which to retrieve the `num_references` value.
- As there is no associated link for the individual, we can safely replace the NaN `num_references` values for rows with extractable names with 0.
- We can proceed with removing the rows that lack an extractable name, as they will not add to the analysis.

In [24]:
# List of rows to keep
keep_rows = [18937, 24985, 27458, 34077, 64771, 106617]

# For loop to replace num_references NaNs with 0 for rows with extractable names
for row in keep_rows:
    df.loc[row, "num_references"] = 0

# List of rows to remove
remove_rows = [index for index in missing_2.index if index not in keep_rows]
del missing_2

# Dropping rows
df.drop(remove_rows, inplace=True)

# Re-checking shape
df.shape

(133900, 6)

<IPython.core.display.Javascript object>

In [25]:
# Re-check info
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133900 entries, 0 to 133904
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   month_year      133900 non-null  object
 1   day             133900 non-null  object
 2   name            133894 non-null  object
 3   info            133900 non-null  object
 4   link            133900 non-null  object
 5   num_references  120634 non-null  object
dtypes: object(6)
memory usage: 7.2+ MB


<IPython.core.display.Javascript object>

#### Observations:
- There are now only 6 rows with missing `name`, corresponding to the names we identified in `info`, that we will extract later.
- The remaining missing values are all for `num_references`, so we can proceed to make another attempt at scraping this information.
- Let us check a sample of these rows.

In [26]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()].sample(5)

,month_year,day,name,info,link,num_references
60579,February 2013,12,Jim Tysinger,", 91, American politician, member of the Georgia Senate (1969–1998), pneumonia.",https://en.wikipedia.org/wiki/Jim_Tysinger,NaN
48966,February 2011,10,Sam Plank,", 62, British radio broadcaster, cancer.",https://en.wikipedia.org/wiki/Sam_Plank,NaN
82681,April 2016,29,Tim Bacon,", 52, British restaurateur and actor.",https://en.wikipedia.org/wiki/Tim_Bacon,NaN
92188,September 2017,24,Al Cannava,", 93, American football player (Green Bay Packers).",https://en.wikipedia.org/wiki/Al_Cannava,NaN
85188,September 2016,18,Robert W. Cone,", 59, American Army general, prostate cancer.",https://en.wikipedia.org/wiki/Robert_W._Cone,NaN


<IPython.core.display.Javascript object>

#### Observations:
- Following the links for each reveals that all 5 pages contain references.  
    https://en.wikipedia.org/wiki/Jim_Tysinger
    
    https://en.wikipedia.org/wiki/Sam_Plank 

    https://en.wikipedia.org/wiki/Tim_Bacon 

    https://en.wikipedia.org/wiki/Al_Cannava  

    https://en.wikipedia.org/wiki/Robert_W._Cone 

- Therefore, they either have a variation in the XPath followed for scraping, or Scrapy had an issue with following their links.
- We will export a dataframe of the pages that need to be re-scraped.

In [38]:
# Exporting dataframe of pages to rescrape for num_references
rescrape_df = df[df["num_references"].isna()]["link"]
rescrape_df.to_csv("rescrape_df.csv", index=False)

<IPython.core.display.Javascript object>